# Dépendance

In [25]:
import seaborn as sns; sns.set()
import pandas as pd
import pandasql as psql
import numpy as np
from sklearn.model_selection import train_test_split as TTS
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.ensemble import RandomForestClassifier as RFC

# Variable

In [2]:
fraud = pd.read_csv('./data/Fraud_Data.csv')
ip = pd.read_csv('./data/IpAddress_to_Country.csv')

# Function

In [3]:
accuracy = lambda p,y: (p==y).sum()/p.shape[0]

In [4]:
def rfc_predict(X,Y,size=0.2, seed=777):
    splits = TTS(X,
                 Y,
                 train_size=size,
                 stratify=Y,
                 random_state=seed
                )
    X_train, X_test, Y_train, Y_test = splits
    rfc = RFC()
    rfc.fit(X_train, Y_train)
    prediction = rfc.predict(X_test)
    print(accuracy(prediction, Y_test))
    print()
    print(rfc.predict_proba(X_test))
    return True


# Notebook

In [5]:
fraud.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0


In [20]:
ip.head()

,lower_bound_ip_address,upper_bound_ip_address,country,ip_address
0,16777216.0,16777471,Australia,0.0
1,16777472.0,16777727,China,0.0
2,16777728.0,16778239,China,0.0
3,16778240.0,16779263,Australia,0.0
4,16779264.0,16781311,China,0.0


In [21]:
fraud = fraud.dropna()
ip = ip.dropna()

In [22]:
#for index, eachf in fraud.iterrows():
#    for _v, te in ip.iterrows():
#        if te.lower_bound_ip_address <= eachf.ip_address <= float(te.upper_bound_ip_address) : 
#            ip.loc[_v, 'ip_address'] = eachf.ip_address
#            print(str(_v) +" : "+ str(ip.iloc[_v]['ip_address']))

query = "SELECT * FROM fraud, ip WHERE fraud.ip_address BETWEEN ip.lower_bound_ip_address AND ip.upper_bound_ip_address"

In [ ]:
data = psql.sqldf(query, locals())
data.head()